In [1]:
import re
import os, glob, datetime, time
import numpy as np
import torch
import torch.nn as nn
from torch.nn.modules.loss import _Loss
import torch.nn.init as init
from torch.utils.data import DataLoader
import torch.optim as optim

from skimage import io, color
from skimage.measure import compare_ssim
from skimage.measure import compare_psnr
import matplotlib.pyplot as plt

# os.environ["CUDA_VISIBLE_DEVICES"] = "3"

In [2]:
gamma = 0.5
sigmaU = 1
sigmaW = 10
epochs = 100

model_name = 'SDN_Color_Block3_1Noise_gamma_%.1f_sigmaU_%.1f_sigmaW_%d'%(gamma, sigmaU, sigmaW)
save_dir = os.path.join('models2', model_name)

In [3]:
class Block3(nn.Module):
    def __init__(self, ch, kernel_size=3):
        super(Block3, self).__init__()
        
        self.conv1 = nn.Sequential(
            nn.Conv2d(ch, ch, kernel_size, padding=1),
            nn.BatchNorm2d(ch),
            nn.ReLU(inplace=True)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(ch, ch, kernel_size, padding=1),
            nn.BatchNorm2d(ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(ch, ch, kernel_size, padding=1),
            nn.BatchNorm2d(ch),
            nn.ReLU(inplace=True)
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(ch, ch, kernel_size, padding=1),
            nn.BatchNorm2d(ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(ch, ch, kernel_size, padding=1),
            nn.BatchNorm2d(ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(ch, ch, kernel_size, padding=1),
            nn.BatchNorm2d(ch),
            nn.ReLU(inplace=True)
        )
    def forward(self, x):
        
        c1 = self.conv1(x)
        c2 = self.conv2(c1+x)
        c3 = self.conv3(c2+x)
        
        return c3

class SDNCNN(nn.Module):
    
    def __init__(self, filters=64, image_channels=3, use_bnorm=True, kernel_size=3):
        super(SDNCNN, self).__init__()
        kernel_size = 3
        padding = 1
        self.conv0 = nn.Conv2d(in_channels=image_channels, out_channels=filters, kernel_size=kernel_size, padding=padding)
        
        self.convOut = nn.Conv2d(in_channels=filters, out_channels=image_channels, kernel_size=kernel_size, padding=padding, bias=True)
        
        self.ResBlock1 = Block3(filters)
        self.ResBlock2 = Block3(filters)
        self.ResBlock3 = Block3(filters)
        self.ResBlock4 = Block3(filters)
        self.ResBlock5 = Block3(filters)
        self.ResBlock6 = Block3(filters)
        self.ResBlock7 = Block3(filters)
        
        self._initialize_weights()

    def forward(self, x):

        c0 = self.conv0(x)
        
        c1 = self.ResBlock1(c0)
        
        c2 = self.ResBlock2(c1)
        
        c3 = self.ResBlock3(c2)
        
        c4 = self.ResBlock4(c3)

        c5 = self.ResBlock5(c4+c3)
        
        c6 = self.ResBlock6(c5+c2)
        
        c7 = self.ResBlock7(c6+c1)
        
        noise = self.convOut(c7)
        
        rec = x - noise
        
        return rec, noise

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                init.orthogonal_(m.weight)
                if m.bias is not None:
                    init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                init.constant_(m.weight, 1)
                init.constant_(m.bias, 0)


def findLastCheckpoint(save_dir):
    file_list = glob.glob(os.path.join(save_dir, 'model_*.pth'))
    if file_list:
        epochs_exist = []
        for file_ in file_list:
            result = re.findall(".*model_(.*).pth.*", file_)
            epochs_exist.append(int(result[0]))
        initial_epoch = max(epochs_exist)
    else:
        initial_epoch = 0
    return initial_epoch

In [4]:
model = SDNCNN()
initial_epoch = findLastCheckpoint(save_dir=save_dir)  # load the last model in matconvnet style
if initial_epoch > 0:
    print('resuming by loading epoch %03d' % initial_epoch)
    model = torch.load(os.path.join(save_dir, 'model_%03d.pth' % initial_epoch))

resuming by loading epoch 118


/home/zhy/anaconda3/lib/python3.7/site-packages/torch/serialization.py:400: UserWarning: Couldn't retrieve source code for container of type DCNN. It won't be checked for correctness upon loading.
  "type " + container_type.__name__ + ". It won't be checked "
/home/zhy/anaconda3/lib/python3.7/site-packages/torch/serialization.py:400: UserWarning: Couldn't retrieve source code for container of type Block3. It won't be checked for correctness upon loading.
  "type " + container_type.__name__ + ". It won't be checked "


In [6]:
with torch.no_grad():
    np.random.seed(42)
    torch.cuda.manual_seed(42)
    torch.manual_seed(42)
    
    output_dir = 'Results_Log/%s'%(model_name)
    output_file_name = 'Results_Log/%s/Log_output_%s.txt'%(model_name, 'CBSD68')
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    test_data = './Test_datasets/CBSD68/*.png'
    test_dir = glob.glob(test_data)
    
    initial_epoch = findLastCheckpoint(save_dir=save_dir)
    for e in range(1, 101):

        model = torch.load(os.path.join(save_dir, 'model_%03d.pth' % e))
        model = model.cuda()
        model.eval()

        psnr_list = []
        ssim_list = []

        for i in range(len(test_dir)):

            start_time = time.time()

            batch_x = io.imread(test_dir[i])
            u = np.random.normal(0, sigmaU, batch_x.shape)
            w = np.random.normal(0, sigmaW, batch_x.shape)
            noise = np.power(batch_x, gamma)*u + w
            batch_y = batch_x + noise

            batch_x = torch.from_numpy(batch_x.transpose(2,0,1).astype('float32'))[None,:,:,:].cuda()
            batch_y = torch.from_numpy(batch_y.transpose(2,0,1).astype('float32'))[None,:,:,:].cuda()

            out, noise_out = model(batch_y)

            batch_x = batch_x.clamp(0, 255)[0,...].cpu().detach().numpy().transpose(1,2,0) /255.0
            out = out.clamp(0, 255)[0,...].cpu().detach().numpy().transpose(1,2,0) /255.0

            psnr_list += [compare_psnr(out, batch_x)]
            
            ssim_list += [compare_ssim(out, batch_x, multichannel=True)]
            
        output_data = 'Epoch: %d, PSNR: %.2f, SSIM: %.4f\n' % (e, np.mean(psnr_list), np.mean(ssim_list))
        output_file = open(output_file_name, 'a')
        output_file.write(output_data)
        output_file.close()

        print('Epoch:%d, PSNR: %.3f, SSIM: %.4f' % (e, np.mean(psnr_list), np.mean(ssim_list)))

Epoch:1, PSNR: 33.531, SSIM: 0.9608
Epoch:2, PSNR: 33.647, SSIM: 0.9612
Epoch:3, PSNR: 33.632, SSIM: 0.9611
Epoch:4, PSNR: 33.598, SSIM: 0.9607
Epoch:5, PSNR: 33.617, SSIM: 0.9608
Epoch:6, PSNR: 33.636, SSIM: 0.9611
Epoch:7, PSNR: 33.622, SSIM: 0.9610
Epoch:8, PSNR: 33.622, SSIM: 0.9609
Epoch:9, PSNR: 33.630, SSIM: 0.9610
Epoch:10, PSNR: 33.569, SSIM: 0.9615
Epoch:11, PSNR: 33.654, SSIM: 0.9626
Epoch:12, PSNR: 33.750, SSIM: 0.9628
Epoch:13, PSNR: 33.686, SSIM: 0.9626
Epoch:14, PSNR: 33.724, SSIM: 0.9624
Epoch:15, PSNR: 33.753, SSIM: 0.9628
Epoch:16, PSNR: 33.761, SSIM: 0.9628
Epoch:17, PSNR: 33.718, SSIM: 0.9626
Epoch:18, PSNR: 33.754, SSIM: 0.9628
Epoch:19, PSNR: 33.705, SSIM: 0.9625
Epoch:20, PSNR: 33.678, SSIM: 0.9625
Epoch:21, PSNR: 33.701, SSIM: 0.9627
Epoch:22, PSNR: 33.696, SSIM: 0.9624
Epoch:23, PSNR: 33.852, SSIM: 0.9640
Epoch:24, PSNR: 33.845, SSIM: 0.9641
Epoch:25, PSNR: 33.865, SSIM: 0.9640
Epoch:26, PSNR: 33.727, SSIM: 0.9637
Epoch:27, PSNR: 33.870, SSIM: 0.9640
Epoch:28, 

In [8]:
with torch.no_grad():
    np.random.seed(42)
    torch.cuda.manual_seed(42)
    torch.manual_seed(42)
    
    output_dir = 'Results_Log/%s'%(model_name)
    output_file_name = 'Results_Log/%s/Log_output_%s.txt'%(model_name, 'kodak')
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    test_data = './Test_datasets/kodak/*.png'
    test_dir = glob.glob(test_data)
    
    initial_epoch = findLastCheckpoint(save_dir=save_dir)
    for e in range(101, initial_epoch):

        model = torch.load(os.path.join(save_dir, 'model_%03d.pth' % e))
        model = model.cuda()
        model.eval()

        psnr_list = []
        ssim_list = []

        for i in range(len(test_dir)):
            start_time = time.time()

            batch_x = io.imread(test_dir[i])
            u = np.random.normal(0, sigmaU, batch_x.shape)
            w = np.random.normal(0, sigmaW, batch_x.shape)
            noise = np.power(batch_x, gamma)*u + w
            batch_y = batch_x + noise
            
            batch_x = torch.from_numpy(batch_x.transpose(2,0,1).astype('float32'))[None,:,:,:].cuda()
            batch_y = torch.from_numpy(batch_y.transpose(2,0,1).astype('float32'))[None,:,:,:].cuda()

            out, noise_out = model(batch_y)
            
            batch_x = batch_x.clamp(0, 255)[0,...].cpu().detach().numpy().transpose(1,2,0) /255.0
            out = out.clamp(0, 255)[0,...].cpu().detach().numpy().transpose(1,2,0) /255.0

            psnr_list += [compare_psnr(out, batch_x)]
            
            ssim_list += [compare_ssim(out, batch_x, multichannel=True)]
            
        output_data = 'Epoch: %d, PSNR: %.2f, SSIM: %.4f\n' % (e, np.mean(psnr_list), np.mean(ssim_list))
        output_file = open(output_file_name, 'a')
        output_file.write(output_data)
        output_file.close()

        print('Epoch:%d, PSNR: %.3f, SSIM: %.4f' % (e, np.mean(psnr_list), np.mean(ssim_list)))

Epoch:101, PSNR: 34.541, SSIM: 0.9630
Epoch:102, PSNR: 34.589, SSIM: 0.9631
Epoch:103, PSNR: 34.594, SSIM: 0.9631
Epoch:104, PSNR: 34.553, SSIM: 0.9631
Epoch:105, PSNR: 34.576, SSIM: 0.9631
Epoch:106, PSNR: 34.591, SSIM: 0.9632
Epoch:107, PSNR: 34.563, SSIM: 0.9631
Epoch:108, PSNR: 34.526, SSIM: 0.9630
Epoch:109, PSNR: 34.579, SSIM: 0.9632
Epoch:110, PSNR: 34.536, SSIM: 0.9631
Epoch:111, PSNR: 34.593, SSIM: 0.9632
Epoch:112, PSNR: 34.598, SSIM: 0.9631
Epoch:113, PSNR: 34.526, SSIM: 0.9629
Epoch:114, PSNR: 34.571, SSIM: 0.9632
Epoch:115, PSNR: 34.572, SSIM: 0.9631
Epoch:116, PSNR: 34.463, SSIM: 0.9630
Epoch:117, PSNR: 34.585, SSIM: 0.9631


In [ ]:
with torch.no_grad():
    
    output_dir = 'Results_Log/%s'%(model_name)
    output_file_name = 'Results_Log/%s/Log_output_%s.txt'%(model_name, 'poly')
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    test_dir = glob.glob('./Test_datasets/poly/real/*.JPG')

    initial_epoch = findLastCheckpoint(save_dir=save_dir)  # load the last model in matconvnet style
    
    for e in range(22, 101):
        model = torch.load(os.path.join(save_dir, 'model_%03d.pth' % e))
        model = model.cuda()
        model.eval()

        psnr_list = []
        ssim_list = []

        for i in range(len(test_dir)):
            mean_name = test_dir[i].split('/')[4].split('.')[0][:-4] + 'mean.JPG'
    #             print(mean_name)
            mean_path = './Test_datasets/poly/mean/%s' % mean_name

            batch_x = io.imread(mean_path)
            batch_y = io.imread(test_dir[i])

            batch_x = torch.from_numpy(batch_x.transpose(2,0,1).astype('float32'))[None,:,:,:].cuda()
            batch_y = torch.from_numpy(batch_y.transpose(2,0,1).astype('float32'))[None,:,:,:].cuda()

            out, noise_out = model(batch_y)

            batch_x = batch_x.clamp(0, 255)[0,...].cpu().detach().numpy().transpose(1,2,0) /255.0
            out = out.clamp(0, 255)[0,...].cpu().detach().numpy().transpose(1,2,0) / 255.0

            psnr_list += [compare_psnr(out, batch_x)]
            ssim_list += [compare_ssim(out, batch_x, multichannel=True)]
            
        output_data = 'Epoch: %d, PSNR: %.2f, SSIM: %.4f\n' % (e, np.mean(psnr_list), np.mean(ssim_list))
        output_file = open(output_file_name, 'a')
        output_file.write(output_data)
        output_file.close()

        print('Epoch: %d, PSNR: %.3f, SSIM: %.4f' % (e, np.mean(psnr_list), np.mean(ssim_list)))

Epoch: 22, PSNR: 37.734, SSIM: 0.9827
Epoch: 23, PSNR: 37.481, SSIM: 0.9824
Epoch: 24, PSNR: 37.460, SSIM: 0.9821
Epoch: 25, PSNR: 37.459, SSIM: 0.9821
Epoch: 26, PSNR: 37.457, SSIM: 0.9819
Epoch: 27, PSNR: 37.467, SSIM: 0.9820
Epoch: 28, PSNR: 37.467, SSIM: 0.9821
Epoch: 29, PSNR: 37.461, SSIM: 0.9820
Epoch: 30, PSNR: 37.506, SSIM: 0.9822
Epoch: 31, PSNR: 37.462, SSIM: 0.9820
Epoch: 32, PSNR: 37.463, SSIM: 0.9821
Epoch: 33, PSNR: 37.489, SSIM: 0.9821
Epoch: 34, PSNR: 37.453, SSIM: 0.9820
Epoch: 35, PSNR: 37.442, SSIM: 0.9819
Epoch: 36, PSNR: 37.462, SSIM: 0.9820
Epoch: 37, PSNR: 37.426, SSIM: 0.9819
Epoch: 38, PSNR: 37.472, SSIM: 0.9822
Epoch: 39, PSNR: 37.437, SSIM: 0.9819
Epoch: 40, PSNR: 37.476, SSIM: 0.9821
Epoch: 41, PSNR: 37.429, SSIM: 0.9819
Epoch: 42, PSNR: 37.445, SSIM: 0.9820
Epoch: 43, PSNR: 37.433, SSIM: 0.9819


In [10]:
with torch.no_grad():

    output_dir = 'Results_Log/%s'%(model_name)
    output_file_name = 'Results_Log/%s/Log_output_%s.txt'%(model_name, 'Nam')
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        
    test_dir = './Test_datasets/Real_ccnoise_denoised_part'
    clean_dir = glob.glob(os.path.join(test_dir, '*mean.png'))
    
    result_dir = './Results/Nam15_gamma_%.1f_sigmaU_%.1f_sigmaW_%d'%(gamma, sigmaU, sigmaW)
    if not os.path.exists(result_dir):
        os.mkdir(result_dir)

    initial_epoch = findLastCheckpoint(save_dir=save_dir)  # load the last model in matconvnet style
    for e in range(1, 101):
        model = torch.load(os.path.join(save_dir, 'model_%03d.pth' % e))
        model = model.cuda()
        model.eval()

        psnr_list = []
        ssim_list = []

        for i in range(len(clean_dir)):
            batch_x = io.imread(clean_dir[i])
            noise_name = clean_dir[i].split('/')[3].split('.')[0][:-4] + 'real.png'
            noise_path = os.path.join(test_dir, noise_name)
            batch_y = io.imread(noise_path)

            batch_x = torch.from_numpy(batch_x.transpose(2,0,1).astype('float32'))[None,:,:,:].cuda()
            batch_y = torch.from_numpy(batch_y.transpose(2,0,1).astype('float32'))[None,:,:,:].cuda()

            out, noise_out = model(batch_y)

            batch_x = batch_x.clamp(0, 255)[0,...].cpu().detach().numpy().transpose(1,2,0) / 255.0
            out = out.clamp(0, 255)[0,...].cpu().detach().numpy().transpose(1,2,0) / 255.0

            psnr_list += [compare_psnr(out, batch_x)]
            ssim_list += [compare_ssim(out, batch_x, multichannel=True)]
            
#             out_name = noise_name[:-8] + 'PSNR_' + '%.2f'%psnr_list[-1] + '_SSIM_' + '%.4f'%ssim_list[-1] + '.png'
#             io.imsave(os.path.join(result_dir, out_name), out)
#             print('%s----%.2f----%.4f'%(noise_name, np.array(psnr_list[-1]), np.array(ssim_list[-1])))

        output_data = 'Epoch: %d, PSNR: %.2f, SSIM: %.4f\n' % (e, np.mean(psnr_list), np.mean(ssim_list))
        output_file = open(output_file_name, 'a')
        output_file.write(output_data)
        output_file.close()
            
            
        print('Epoch: %d, PSNR: %.3f, SSIM: %.4f' % (e, np.mean(psnr_list), np.mean(ssim_list)))

Epoch: 1, PSNR: 36.622, SSIM: 0.9732
Epoch: 2, PSNR: 36.589, SSIM: 0.9732
Epoch: 3, PSNR: 36.588, SSIM: 0.9732
Epoch: 4, PSNR: 36.615, SSIM: 0.9735
Epoch: 5, PSNR: 36.585, SSIM: 0.9733
Epoch: 6, PSNR: 36.581, SSIM: 0.9731
Epoch: 7, PSNR: 36.594, SSIM: 0.9733
Epoch: 8, PSNR: 36.581, SSIM: 0.9732
Epoch: 9, PSNR: 36.592, SSIM: 0.9732
Epoch: 10, PSNR: 36.696, SSIM: 0.9737
Epoch: 11, PSNR: 36.850, SSIM: 0.9745
Epoch: 12, PSNR: 36.863, SSIM: 0.9747
Epoch: 13, PSNR: 36.882, SSIM: 0.9749
Epoch: 14, PSNR: 36.881, SSIM: 0.9750
Epoch: 15, PSNR: 36.880, SSIM: 0.9748
Epoch: 16, PSNR: 36.862, SSIM: 0.9746
Epoch: 17, PSNR: 36.899, SSIM: 0.9750
Epoch: 18, PSNR: 36.873, SSIM: 0.9747
Epoch: 19, PSNR: 36.885, SSIM: 0.9749
Epoch: 20, PSNR: 36.867, SSIM: 0.9748
Epoch: 21, PSNR: 36.869, SSIM: 0.9747
Epoch: 22, PSNR: 36.890, SSIM: 0.9750
Epoch: 23, PSNR: 36.822, SSIM: 0.9745
Epoch: 24, PSNR: 36.812, SSIM: 0.9743
Epoch: 25, PSNR: 36.812, SSIM: 0.9743
Epoch: 26, PSNR: 36.785, SSIM: 0.9741
Epoch: 27, PSNR: 36.8